In [1]:
"""
Data augmentation using TorchIO.
Link: https://torchio.readthedocs.io/index.html

We will upload each volume, apply transformation and save resulting volumes on the disk.
Transformations are applied to both volume and the mask.
"""

'\nData augmentation using TorchIO.\nLink: https://torchio.readthedocs.io/index.html\n\nWe will upload each volume, apply transformation and save resulting volumes on the disk.\nTransformations are applied to both volume and the mask.\n'

In [41]:
# Import modules
import os, glob
import torchio as tio
from tqdm.notebook import tqdm, tnrange

In [42]:
# Define paths
# Since data size is huge, I am using external drive to save augmented data

data_dir = os.path.abspath("/media/trojan/evo/3D-CT-Artery-Segmentation/data/train")
volumes_dir = os.path.join(data_dir + "/volumes")
masks_dir = os.path.join(data_dir + "/masks")
volumes_paths = sorted(glob.glob(volumes_dir + "/" + "*.nii.gz"))
masks_paths = sorted(glob.glob(masks_dir + "/" + "*.nii.gz"))
out_vol_dir = "/media/trojan/evo/3D-CT-Artery-Segmentation/aug_data/volumes"
out_mask_dir = "/media/trojan/evo/3D-CT-Artery-Segmentation/aug_data/masks"

In [43]:
print(volumes_paths)

['/media/trojan/evo/3D-CT-Artery-Segmentation/data/train/volumes/1.nii.gz', '/media/trojan/evo/3D-CT-Artery-Segmentation/data/train/volumes/10.nii.gz', '/media/trojan/evo/3D-CT-Artery-Segmentation/data/train/volumes/11.nii.gz', '/media/trojan/evo/3D-CT-Artery-Segmentation/data/train/volumes/12.nii.gz', '/media/trojan/evo/3D-CT-Artery-Segmentation/data/train/volumes/13.nii.gz', '/media/trojan/evo/3D-CT-Artery-Segmentation/data/train/volumes/14.nii.gz', '/media/trojan/evo/3D-CT-Artery-Segmentation/data/train/volumes/15.nii.gz', '/media/trojan/evo/3D-CT-Artery-Segmentation/data/train/volumes/16.nii.gz', '/media/trojan/evo/3D-CT-Artery-Segmentation/data/train/volumes/17.nii.gz', '/media/trojan/evo/3D-CT-Artery-Segmentation/data/train/volumes/2.nii.gz', '/media/trojan/evo/3D-CT-Artery-Segmentation/data/train/volumes/3.nii.gz', '/media/trojan/evo/3D-CT-Artery-Segmentation/data/train/volumes/4.nii.gz', '/media/trojan/evo/3D-CT-Artery-Segmentation/data/train/volumes/6.nii.gz', '/media/trojan/e

In [44]:
# Load the volumes
subjects = {}
for (volume_path, mask_path) in zip(volumes_paths, masks_paths):
    subject = tio.Subject(
        vol=tio.ScalarImage(volume_path),
        mask=tio.LabelMap(mask_path),
    )
    _id = volume_path.split('/')[-1].split('.')[0]
    subjects[_id]=subject
dataset = tio.SubjectsDataset(subjects.values())
print('Dataset size:', len(dataset), 'subjects')

Dataset size: 16 subjects


In [45]:
"""
Reference Link: https://torchio.readthedocs.io/transforms/augmentation.html#augmentation
Check the link for more details.
"""

# Define transformations inside dicts
transform_dict = {
    tio.RandomBiasField(),
    tio.RandomNoise(std=(0,100)),
    tio.RandomFlip(axes=(0,1), flip_probability=1),
    tio.RandomGhosting(),
    tio.RandomSpike(),
    tio.OneOf({
        tio.RandomAffine(
            scales=(0.9, 1.2),
            degrees=(15, 15)
        ): 0.8,
        tio.RandomElasticDeformation(): 0.2,
    }),
} 

# Composed transformation to apply multiple transformations at once
transform_compose = tio.Compose([
    tio.RandomBiasField(p=0.2),
    tio.RandomNoise(std=(0,100)),
    tio.RandomFlip(p=0.2, axes=(0,1), flip_probability=1),
    tio.RandomGhosting(p=0.2),
    tio.RandomSpike(p=0.2),
    tio.OneOf(p=0.2, transforms = {
        tio.RandomAffine(
            scales=(0.9, 1.2),
            degrees=(15, 15)
        ): 0.8,
        tio.RandomElasticDeformation(): 0.2,
    }),
])

In [46]:
# Define the final transform function
# We will apply compose transformation with 20% probability
# And single transformations with 80%

# transform = tio.OneOf(transform_dict)

In [47]:
i = 1
for transform in tqdm(transform_dict):
    for idx, sample in tqdm(subjects.items()):
        output = transform(sample)
        temp = list(output.values())
        temp[0].save(os.path.join(out_vol_dir, f"{idx}_{i}.nii.gz"))
        temp[1].save(os.path.join(out_mask_dir, f"{idx}_{i}.nii.gz"))
    i+=1

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

In [34]:
for idx, sample in tqdm(subjects.items()):
    for i in tnrange(1, 6):
        output = transform(sample)
        temp = list(output.values())
        temp[0].save(os.path.join(out_vol_dir, f"{idx}_{i}.nii.gz"))
        temp[1].save(os.path.join(out_mask_dir, f"{idx}_{i}.nii.gz"))

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

KeyboardInterrupt: 

While running the `augmentation` process, the file `5.nii.gz` gave an error because of size mismatch between `volume` and the `mask`.
Not sure which volume is the correct one and was used for labeling originally. So excluding this file from training.

Had to run it again on the remaining files.

In [ ]:
################# Delete after completion ###########################3

In [3]:
# Define paths
# Since data size is huge, I am using external drive to save augmented data

data_dir = os.path.abspath("./data/train")
volumes_dir = os.path.join(data_dir + "/volumes")
masks_dir = os.path.join(data_dir + "/masks")
volumes_paths = sorted(glob.glob(volumes_dir + "/" + "*.nii.gz"))
masks_paths = sorted(glob.glob(masks_dir + "/" + "*.nii.gz"))
out_vol_dir = "/run/user/1000/gvfs/afp-volume:host=SKIA.local,user=rajahaseeb147,volume=Shared/haseeb/3D-CT-Artery-Segmentation/aug_data/volumes"
out_mask_dir = "/run/user/1000/gvfs/afp-volume:host=SKIA.local,user=rajahaseeb147,volume=Shared/haseeb/3D-CT-Artery-Segmentation/aug_data/masks"

In [6]:
print(volumes_paths[-4:])

['/home/trojan/skia_projects/3D-CT-Artery-Segmentation/data/train/volumes/6.nii.gz', '/home/trojan/skia_projects/3D-CT-Artery-Segmentation/data/train/volumes/7.nii.gz', '/home/trojan/skia_projects/3D-CT-Artery-Segmentation/data/train/volumes/8.nii.gz', '/home/trojan/skia_projects/3D-CT-Artery-Segmentation/data/train/volumes/9.nii.gz']


In [8]:
print(masks_paths[-4:])

['/home/trojan/skia_projects/3D-CT-Artery-Segmentation/data/train/masks/6.nii.gz', '/home/trojan/skia_projects/3D-CT-Artery-Segmentation/data/train/masks/7.nii.gz', '/home/trojan/skia_projects/3D-CT-Artery-Segmentation/data/train/masks/8.nii.gz', '/home/trojan/skia_projects/3D-CT-Artery-Segmentation/data/train/masks/9.nii.gz']


In [9]:
# Load the volumes
subjects = {}
for (volume_path, mask_path) in zip(volumes_paths[-4:], masks_paths[-4:]):
    subject = tio.Subject(
        vol=tio.ScalarImage(volume_path),
        mask=tio.LabelMap(mask_path),
    )
    _id = volume_path.split('/')[-1].split('.')[0]
    subjects[_id]=subject
dataset = tio.SubjectsDataset(subjects.values())
print('Dataset size:', len(dataset), 'subjects')

Dataset size: 4 subjects


In [10]:
"""
Reference Link: https://torchio.readthedocs.io/transforms/augmentation.html#augmentation
Check the link for more details.
"""

# Define transformations inside dicts
transform_dict = {
    #tio.Resample(4), # Don't use this
    tio.RandomMotion(),
    tio.RandomBiasField(),
    tio.RandomNoise(std=(0,400)),
    tio.RandomFlip(axes=(0,1), flip_probability=1),
    tio.RandomAnisotropy(axes=(0,1,2), downsampling=6),
    tio.RandomGhosting(),
    tio.RandomSpike(),
    tio.RandomBlur(std=(0,3)),
    tio.RandomSwap(patch_size=30),
    tio.RandomGamma(log_gamma=(-0.5, 0.5)),
    tio.OneOf({
        tio.RandomAffine(
            scales=(0.9, 1.2),
            degrees=(15, 15)
        ): 0.8,
        tio.RandomElasticDeformation(): 0.2,
    }),
} 

# Composed transformation to apply multiple transformations at once
transform_compose = tio.Compose([
    #tio.Resample(p=0.3, target=3),
    tio.RandomMotion(p=0.2),
    tio.RandomBiasField(p=0.3),
    tio.RandomNoise(p=0.5, std=(0,400)),
    tio.RandomFlip(axes=(0,1), flip_probability=0.5),
    tio.RandomAnisotropy(p=0.3, axes=(0,1,2), downsampling=6),
    tio.RandomGhosting(p=0.2),
    tio.RandomSpike(p=0.3),
    tio.RandomBlur(p=0.3, std=(0,3)),
    tio.RandomSwap(p=0.2, patch_size=30),
    tio.RandomGamma(p=0.3, log_gamma=(-0.5, 0.5)),
    tio.OneOf(p=0.3, transforms = {
        tio.RandomAffine(
            scales=(0.9, 1.2),
            degrees=(15, 15)
        ): 0.8,
        tio.RandomElasticDeformation(): 0.2,
    }),
])

In [11]:
# Define the final transform function
# We will apply compose transformation with 20% probability
# And single transformations with 80%

transform = tio.OneOf({transform_compose: 0.2, tio.OneOf(transform_dict): 0.8})

In [12]:
for idx, sample in subjects.items():
    for i in range(1, 51):
        output = transform(sample)
        temp = list(output.values())
        temp[0].save(os.path.join(out_vol_dir, f"{idx}_{i}.nii.gz"))
        temp[1].save(os.path.join(out_mask_dir, f"{idx}_{i}.nii.gz"))

In [ ]:
#########################################3